In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Part 1

In [35]:
class Brick:
    def __init__(self,line):
        line = line.strip()
        start,end = line.split('~')
        
        self.start = [ int(i) for i in start.split(',')]
        self.end = [ int(i) for i in end.split(',')]
        self.idx = None
    
    def __repr__(self):
        return f'{self.start} to {self.end}'
    
    
    def put_brick_grid(self, grid):
        if self.idx is None: 
            val = grid.max() +1
            self.idx = val
        else:
            val = self.idx
        
        for x in range(self.start[0],self.end[0]+1):
            for y in range(self.start[1],self.end[1]+1):
                for z in range(self.start[2],self.end[2]+1):
                    grid[x][y][z] = val
                    
        return grid
            
        

In [160]:
def drop_bricks(bricks,grid):
    bidx =0
    while bidx < len(bricks):
        b = bricks[bidx]
        
        z_below = b.start[2]-1
        xs,ys = b.start[:2]
        xe,ye = b.end[:2]
        slide =0
        while grid[xs:xe+1,ys:ye+1,z_below+slide].sum() == 0:
            slide += -1
        
        if slide != 0 :
            b.start[2] += slide
            b.end[2] += slide
            grid = np.multiply((grid != b.idx).astype(int),grid)
            b.put_brick_grid(grid)
            bidx = 0
        else:
            bidx += 1
            
    return bricks, grid

In [172]:
def nonsupport_blocks(bricks, grid):
    possible = [ i for i in range(1,int(grid.max())+1)]
    
    for b in bricks:
        
        z_below = b.start[2]-1
        xs,ys = b.start[:2]
        xe,ye = b.end[:2]
        sub_grid = grid[xs:xe+1,ys:ye+1,z_below]
        under = set(sub_grid[sub_grid>0].ravel())
        if len(under) == 1:
            val = list(under)[0]
            if val in possible:
                possible.remove(val)
                
    return possible

In [175]:
import numpy as np
def do_part_one(file_path):
    xmax, ymax, zmax = 0,0,0
    
    with open(file_path,'r') as f:
        bricks = [Brick(l)  for l in f.readlines()]
        
    for b in bricks:
        xmax = max(xmax, b.end[0])
        ymax = max(ymax, b.end[1])
        zmax = max(zmax, b.end[2])
    
    grid = np.zeros((xmax+1, ymax+1, zmax+1))
    for x in range(0,xmax+1):
        for y in range(0,ymax+1):
            grid[x,y,0] = -1 
    
    
    for b in bricks:
        b.put_brick_grid(grid)
    bricks, grid = drop_bricks(bricks, grid)
    
    pos = nonsupport_blocks(bricks, grid)
    
    return len(pos)

In [176]:
%%time
do_part_one('input_data/test_22.txt')


CPU times: user 1.42 ms, sys: 1.08 ms, total: 2.5 ms
Wall time: 1.62 ms


5

In [177]:
%%time
do_part_one('input_data/day_22.txt')

CPU times: user 6.1 s, sys: 3.52 ms, total: 6.1 s
Wall time: 6.11 s


389

# Part 2

In [195]:
from copy import deepcopy

def disintegrate_brick(orig_bricks,orig_grid, bi):
    val= orig_bricks[bi].idx
    bricks = deepcopy(orig_bricks[:bi] + orig_bricks[bi+1:])
    grid = deepcopy(np.multiply((orig_grid != val).astype(int),orig_grid))
    
    bidx =0
    moved =[]
    while bidx < len(bricks):
        b = bricks[bidx]
        
        z_below = b.start[2]-1
        xs,ys = b.start[:2]
        xe,ye = b.end[:2]
        slide =0
        while grid[xs:xe+1,ys:ye+1,z_below+slide].sum() == 0:
            slide += -1
        
        if slide != 0 :
            moved.append(bidx)
            b.start[2] += slide
            b.end[2] += slide
            grid = np.multiply((grid != b.idx).astype(int),grid)
            b.put_brick_grid(grid)
            bidx = 0
        else:
            bidx += 1
            
    return len(set(moved))

In [196]:
def do_part_two(file_path):
    xmax, ymax, zmax = 0,0,0
    
    with open(file_path,'r') as f:
        bricks = [Brick(l)  for l in f.readlines()]
        
    for b in bricks:
        xmax = max(xmax, b.end[0])
        ymax = max(ymax, b.end[1])
        zmax = max(zmax, b.end[2])
    
    grid = np.zeros((xmax+1, ymax+1, zmax+1))
    for x in range(0,xmax+1):
        for y in range(0,ymax+1):
            grid[x,y,0] = -1 
    
    
    for b in bricks:
        b.put_brick_grid(grid)
    bricks, grid = drop_bricks(bricks, grid)
    
    total = sum([ disintegrate_brick(bricks,grid, bi) for bi in range(len(bricks))]) 
    return total

In [197]:
%%time
do_part_two('input_data/test_22.txt')
           

CPU times: user 4.64 ms, sys: 2.02 ms, total: 6.66 ms
Wall time: 5.6 ms


7

In [198]:
%%time
do_part_two('input_data/day_22.txt')
# 88851 is too high

CPU times: user 1min 23s, sys: 102 ms, total: 1min 23s
Wall time: 1min 23s


70609